In [8]:
import numpy as np
from scipy.stats import beta

# parameters and model set up

sigma = 5               # elasticity of substitution between critical goods
theta = 0.1             # between outside and critical good
kappa = 0.05                # fixed entry cost
z_h = {"A": 0.0, "B": 0.0}  # production subsidies for countries A and B
e_h = {"A": 0.0, "B": 0.0}  # entry subsidy for countries A and B
tau = 0.1                   # generic trade cost

imtax_Fij = {"A": {"A": 0.00, "B": 0.00, "C": 0.00}, # import tariff on critical goods from i to j
             "B": {"A": 0.00, "B": 0.00, "C": 0.00}}

extax_Fij = {"A": {"A": 0.00, "B": 0.00}, # export tax on critical goods from i to j
             "B": {"A": 0.00, "B": 0.00}}

imtax_Ihi = {"A": {"A": 0.00, "B": 0.00}, # import tariff on inputs from h to i
             "B": {"A": 0.00, "B": 0.00}}

extax_Ihi = {"A": {"A": 0.00, "B": 0.00}, # export tax on inputs from h to i
             "B": {"A": 0.00, "B": 0.00}}

alpha_c = {"A": 19, "B": 19}  # alpha for country risk, e.g. 6 (~86% survival), 9 (90%), 19 (95%), 99 (99%)
beta_c = {"A": 1, "B": 1}     # beta parameter (fix to 1)
alpha_i = 6                   # alpha parameter for idiosyncratic risk
beta_i = 1

cvar_alpha = 0.95  # confidence level for CVaR

np.random.seed(42) # random number seed for reproducibility

In [ ]:
import math
import pandas as pd

# high barrier: kappa = 0.1 (alpha_a = 19, b = 19, i = 6)
# low barrier: kappa = 0.05

kappa = 0.1

# monte carlo setup
num_simulations = 40000
max_iterations = 1000
min_iterations = 5 # min before checking convergence
tolerance = 0.01   # relative error tolerance
z_alpha = 1.96     # critical z-value for 95% confidence

# define the logging function
def log_progress(current, total, interval=1):
    """Logs progress at a specified interval."""
    if current % interval == 0 or current == total:
        print(f"Processed {current} out of {total} ({(current / total) * 100:.2f}%)")

# expected survival probability
expected_phi_h = {
    "A": (alpha_c["A"]/(alpha_c["A"] + beta_c["A"]))*(alpha_i/(alpha_i + beta_i)),
    "B": (alpha_c["B"]/(alpha_c["B"] + beta_c["B"]))*(alpha_i/(alpha_i + beta_i)),
}

# generate sequence of subsidy values
sub_sequence = np.arange(0, 0.55, 0.05)
esub_loop_results = []

# count total iterations for logging
total_iterations = len(sub_sequence) ** 2
current_iteration = 0

for i in sub_sequence:

    # choose entry subsidy or production subsidy by commenting out
    # e_A = i
    z_A = i
    
    for j in sub_sequence:

        # e_B = j
        z_B = j
        
        # e_h = {"A": e_A, "B": e_B}
        z_h = {"A": z_A, "B": z_B}
        
        # calculate input prices
        p_Ih = {
            "A": (sigma / (sigma - 1)) * (1 / expected_phi_h["A"] - z_h["A"]),
            "B": (sigma / (sigma - 1)) * (1 / expected_phi_h["B"] - z_h["B"]),
        }
        
        # trade costs for critical goods from A to (A, B)
        t_Fij = {
            i: {
                j: 1 if i == j else 1 + imtax_Fij.get(i, {}).get(j, 0) + extax_Fij.get(i, {}).get(j, 0) + tau
                for j in ("A", "B", "C") if j in imtax_Fij.get(i, {})
            }
            for i in ("A", "B", "C") if i in imtax_Fij
        }
        
        # trade costs for critical goods from A to (A, B)
        t_Ihi = {
            h: {
                i: 1 if h == i else 1 + imtax_Ihi.get(h, {}).get(i, 0) + extax_Ihi.get(h, {}).get(i, 0) + tau
                for i in ("A", "B", "C") if i in imtax_Ihi.get(h, {})
            }
            for h in ("A", "B") if h in imtax_Ihi
        }
        
        # calculate price of final goods (p_Fi) = input price index (bar_P_Ii)
        p_Fi = {i: np.sum([(p_Ih[h] * t_Ihi[h][i]) ** (1 - sigma) for h in ("A", "B")]) ** (1 / (1 - sigma)) for i in ("A", "B")}
        
        # calculate price index for final goods consumed in country j
        bar_P_Fj = {j: np.sum([(p_Fi[i] * t_Fij[i][j]) ** (1 - sigma) for i in ("A", "B")]) ** (1 / (1 - sigma)) for j in ("A", "B", "C")}
        
        # expected demand for final goods
        expected_x_Fij = {
            i: {j: (p_Fi[i] * t_Fij[i][j]) ** (-sigma) * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta)) for j in ("A", "B", "C")}
            for i in ("A", "B")
        }
        
        # expected demand for inputs
        expected_X_Ihi = {
            h: {
                i: (p_Ih[h] * t_Ihi[h][i]) ** -sigma
                * sum(
                    t_Fij[i][j] ** -sigma
                    * bar_P_Fj[j] ** ((sigma * (1 - theta) - 1) / (1 - theta))
                    for j in ("A", "B", "C")
                )
                for i in ("A", "B")
            }
            for h in ("A", "B")
        }
        
        # n_h (number of entrants) for each h, rounded down
        n_h = {
            h: math.floor(
                sum(expected_X_Ihi[h][i] for i in ("A", "B"))
                / ((sigma - 1) * (1 - e_h[h]) * kappa)
                * ((1 / expected_phi_h[h]) - z_h[h])
            )
            for h in ("A", "B")
        }
        
        # monte carlo setup
        iteration = 0
        results = []
        cvar_results = []
                
        # run the loop until convergence or max iterations
        while iteration < max_iterations:
            iteration += 1   
            
            # generate risk factors
            D_h = {
                h: 1 - beta.rvs(alpha_c[h], beta_c[h], size = num_simulations)
                for h in ("A", "B")
            }
            
            fail_all_h = {
                h: np.random.rand(num_simulations) < D_h[h]
                for h in ("A", "B")
            }
        
            # get fraction of surviving firms for each sample
            # number of survivors
            ns_h = {
                h: [
                    0 if fail_all_h[h][sim] >= 1 else np.random.binomial(n_h[h], beta.rvs(alpha_i, beta_i))
                    for sim in range(num_simulations)
                ]
                for h in ("A", "B")
            }
            # fraction (survivors over entrants)
            fraction_survived_h = {
                h: [0 if n_h[h] == 0 else ns / n_h[h] for ns in ns_h[h]]
                for h in ("A", "B")
            }
          
            # compute rho_i directly for all samples
            rho_i = {
                i: (
                    np.sum(
                        [
                            (
                                np.array(fraction_survived_h[h]) / expected_phi_h[h]
                            ) ** ((sigma - 1) / sigma)
                            * (
                                (1 / expected_phi_h[h] - z_h[h]) ** (1 - sigma)
                            )
                            * (t_Ihi[h][i] ** (1 - sigma))
                            for h in ["A", "B"]
                        ],
                        axis=0,  # sum across the simulations
                    )
                    / np.sum(
                        [
                            (
                                (1 / expected_phi_h[h] - z_h[h]) ** (1 - sigma)
                                * (t_Ihi[h][i] ** (1 - sigma))
                            )
                            for h in ["A", "B"]
                        ],
                        axis=0,  # sum across the simulations
                    )
                )
                ** (sigma / (sigma - 1))
                for i in ["A", "B"]
            }

            # tariff revenue calculation
            tariff_revenue = {}
            
            for j in ("A", "B", "C"):
                revenue = 0
            
                # import tariff revenue on final goods
                revenue += sum(
                    imtax_Fij.get(i, {}).get(j, 0) * rho_i[i] * expected_x_Fij[i][j]
                    for i in ("A", "B") if i != j
                )
            
                # import tariff revenue on inputs
                if j in ("A", "B"):
                    revenue += sum(
                        imtax_Ihi.get(i, {}).get(j, 0) * fraction_survived_h[i][iteration] / 
                        expected_phi_h[i] * expected_X_Ihi[i][j]
                        for i in ("A", "B") if i != j
                    )
            
                # export tax revenue on final goods
                if j in ("A", "B"):
                    revenue += sum(
                        extax_Fij.get(j, {}).get(i, 0) * rho_i[j] * expected_x_Fij[j][i]
                        for i in ("A", "B") if j != i
                    )
            
                # export tax revenue on inputs
                if j in ("A", "B"):
                    revenue += sum(
                        extax_Ihi.get(j, {}).get(i, 0) * fraction_survived_h[j][iteration] / 
                        expected_phi_h[j] * expected_X_Ihi[j][i]
                        for i in ("A", "B") if j != i
                    )
            
                tariff_revenue[j] = revenue
            
            # production/entry subsidy cost
            subsidy_cost = {}
            
            for h in ("A", "B"):
                # entry subsidy cost
                entry_subsidy = fraction_survived_h[h][iteration] * n_h[h] * e_h[h] * kappa
                
                # production subsidy cost
                production_subsidy = z_h[h] * sum(
                    fraction_survived_h[h][iteration] / expected_phi_h[h] * expected_X_Ihi[h][i]
                    for i in ("A", "B")
                )
                
                # total subsidy cost
                subsidy_cost[h] = entry_subsidy + production_subsidy

            # profits in each country
            profits = {}
            
            for h in ("A", "B"):
                # calculate profit components for each country
                production_profit = sum(
                    fraction_survived_h[h][iteration] / expected_phi_h[h] * expected_X_Ihi[h][i] *
                    (p_Ih[h] + z_h[h] - 1 / expected_phi_h[h])
                    for i in ("A", "B")
                )
                fixed_costs = n_h[h] * (1 - e_h[h]) * kappa
                
                # total profits
                profits[h] = production_profit - fixed_costs
            
            # outside good consumption
            x_0j = {
                j: 1
                + tariff_revenue.get(j, 0)
                - subsidy_cost.get(j, 0)
                + profits.get(j, 0)
                - bar_P_Fj.get(j, 0) ** (-theta / (1 - theta))
                for j in ("A", "B", "C")
            }
            
            # compute utility U_j
            sum_term = {
                j: 
                (((rho_i["A"] * expected_x_Fij["A"][j]) ** ((sigma - 1) / sigma) + 
                  (rho_i["B"] * expected_x_Fij["B"][j]) ** ((sigma - 1) / sigma)))
                for j in ("A", "B", "C")
            }
            
            U_j = {
                j: x_0j.get(j, 0) +
                   (1 / theta) * (sum_term.get(j, 0) ** (theta * sigma / (sigma - 1)))
                for j in ("A", "B")
            }
            
            # calculate VaR as a dictionary
            var_threshold = {
                j: np.percentile(U_j[j], (1 - cvar_alpha) * 100) for j in U_j.keys()
            }
            
            # calculate CVaR as a dictionary
            cvar = {
                j: np.mean(U_j[j][U_j[j] <= var_threshold[j]]) for j in U_j.keys()
            }
            
            # append results
            cvar_results.append(cvar)
            results.append({j: np.mean(U_j[j]) for j in U_j.keys()})
        
            # extract utility and CVaR results for country "A"
            cvar_results_A = [cvar["A"] for cvar in cvar_results]
            results_A = [result["A"] for result in results]
            
            # calculate standard errors for country "A"
            cvar_sem_A = (
                np.std(cvar_results_A, ddof=1) / np.sqrt(len(cvar_results_A))
                if len(cvar_results_A) > 1
                else 0  # Use 0 if variance can't be computed
            )
            mean_sem_A = (
                np.std(results_A, ddof=1) / np.sqrt(len(results_A))
                if len(results_A) > 1
                else 0
            )
            
            # Calculate confidence interval widths for country "A"
            mean_ci_width_A = 2 * z_alpha * mean_sem_A
            cvar_ci_width_A = 2 * z_alpha * cvar_sem_A
            
            # Check convergence for country "A"
            mean_converged_A = (
                len(results_A) > min_iterations
                and (mean_ci_width_A / np.abs(np.mean(results_A))) < tolerance
            )
            cvar_converged_A = (
                len(cvar_results_A) > min_iterations
                and (cvar_ci_width_A / np.abs(np.mean(cvar_results_A))) < tolerance
            )
            
            if mean_converged_A and cvar_converged_A:
                break
        else:
            print("Reached maximum iterations without convergence.")
        
        # after the monte carlo loop
        final_cvar = {k: np.mean([c[k] for c in cvar_results]) for k in cvar.keys()}
        final_utility = {k: np.mean([r[k] for r in results]) for k in results[0].keys()}

        # log progress
        current_iteration += 1
        log_progress(current_iteration, total_iterations, interval = 10)
               
        # append to ex_loop_results
        esub_loop_results.append({
            # "e_A": e_A,
            # "e_B": e_B,
            "z_A": z_A,
            "z_B": z_B,
            "utility_A": final_utility["A"],
            "utility_B": final_utility["B"],
            "cvar_A": final_cvar["A"],
            "cvar_B": final_cvar["B"],
            "entrants_A": n_h["A"],
            "entrants_B": n_h["B"]
        })

# save csv
results_df = pd.DataFrame(esub_loop_results)
results_df.to_csv("output/discrete_production_subsidy_loop_high_kappa.csv", index = False)

print("Results saved as discrete_production_subsidy_loop_high_kappa.csv")

Processed 10 out of 121 (8.26%)
Processed 20 out of 121 (16.53%)
Processed 30 out of 121 (24.79%)
Processed 40 out of 121 (33.06%)
Processed 50 out of 121 (41.32%)
Processed 60 out of 121 (49.59%)
Processed 70 out of 121 (57.85%)
Processed 80 out of 121 (66.12%)
Processed 90 out of 121 (74.38%)
Reached maximum iterations without convergence.
Processed 100 out of 121 (82.64%)
Reached maximum iterations without convergence.
Processed 110 out of 121 (90.91%)
Reached maximum iterations without convergence.
Reached maximum iterations without convergence.


In [18]:
results_df = pd.DataFrame(esub_loop_results)
results_df

,z_A,z_B,utility_A,utility_B,cvar_A,cvar_B
0,0.0,0.00,9.792084,9.812606,8.418594,8.439616
1,0.0,0.05,9.776299,9.744764,8.388264,8.304120
2,0.0,0.10,9.843065,9.691750,8.452509,8.040856
3,0.0,0.15,9.823777,9.663777,8.159671,7.923791
4,0.0,0.20,9.842695,9.547872,7.555688,7.196860
5,0.0,0.25,9.881410,9.486282,7.471304,6.712863
6,0.0,0.30,9.871998,9.337058,7.185240,6.427017
7,0.0,0.35,9.898727,9.195793,6.912888,6.112796
